# Microgrid sizing optimization: compare algo record learning curves

Notebook extended from the **Microgrid sizing optimization** notebook example in `../Microgrids.jl/examples/`.

PH, sept 2023

In [22]:
using Microgrids
using NLopt # optimization solvers
using Printf # pretty print results
using Random

## Load Microgrid project data

Loading parameters and time series for a Microgrid project with *wind* and *solar* sources, plus a *battery* and a *dispatchable generator*. 
Values gathered from the Microgrid_Wind-Solar.ipynb example notebook.

In [2]:
include("../Microgrids.jl/examples/Microgrid_Wind-Solar_data.jl");

Data definition for Microgrid with wind, solar, storage and generator...


## Setting up the cost function (criterion) to be optimized


In [3]:
"""Simulate the performance of a Microgrid project of size `x`
with x=[power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind]

Returns stats, costs
"""
    function simulate_microgrid(x)
    project = Project(lifetime, discount_rate, timestep, "€")
    # Split decision variables (converted MW → kW):
    power_rated_gen = x[1]*1000
    energy_rated_sto = x[2]*1000
    power_rated_pv = x[3]*1000
    power_rated_wind = x[4]*1000

    # Create components
    gen = DispatchableGenerator(power_rated_gen,
        fuel_intercept, fuel_slope, fuel_price,
        investment_price_gen, om_price_gen, lifetime_gen,
        load_ratio_min,
        replacement_price_ratio, salvage_price_ratio, fuel_unit)
    batt = Battery(energy_rated_sto,
        investment_price_sto, om_price_sto, lifetime_sto, lifetime_cycles,
        charge_rate, discharge_rate, loss_factor_sto, SoC_min, SoC_ini,
        replacement_price_ratio, salvage_price_ratio)
    pv = Photovoltaic(power_rated_pv, irradiance,
        investment_price_pv, om_price_pv,
        lifetime_pv, derating_factor_pv,
        replacement_price_ratio, salvage_price_ratio)
    windgen = WindPower(power_rated_wind, cf_wind,
        investment_price_wind, om_price_wind,
        lifetime_wind,
        replacement_price_ratio, salvage_price_ratio)
    mg = Microgrid(project, Pload, gen, batt, [pv, windgen])

    # Launch simulation:
    traj, stats, costs = simulate(mg)

    return stats, costs
end

simulate_microgrid

In [4]:
"Multi-objective criterion for microgrid performance: lcoe, shedding rate"
function obj_multi(x)
    stats, costs = simulate_microgrid(x)
    # Extract KPIs of interest
    lcoe = costs.lcoe # $/kWh
    shed_rate = stats.shed_rate; # in [0,1]
    return lcoe, shed_rate
end

obj_multi

In [5]:
"""Mono-objective criterion: LCOE + penalty if shedding rate > `shed_max`

with signature adapted to NLopt with `grad` as 2nd argument

load shedding penalty threshold `shed_max` should be in  [0,1[
"""
function obj(x, grad, shed_max, w_shed_max=1e5)
    lcoe, shed_rate = obj_multi(x)
    over_shed = shed_rate - shed_max
    if over_shed > 0.0
        penalty = w_shed_max*over_shed
    else
        penalty = 0.0
    end
    J = lcoe + penalty
end

obj

### Tests the objective functions

Test of the simulator wrapper (on a baseline sizing):

In [6]:
# Baseline sizing: same as in Microgrid_Wind-Solar.ipynb notebook
x_base = [power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind]/1000.
# run simulation:
stats, costs = simulate_microgrid(x_base)
x_base, costs.lcoe, costs.npc/1e6

([1.8, 5.0, 3.0, 0.9], 0.22924812869928668, 21.89002772908652)

In [7]:
# Test:
shed_max = 0.01 # in [0,1]

println("Base. multi: ", obj_multi(x_base), " mono: ", obj(x_base,[], shed_max))

Base. multi: (0.22924812869928668, 0.0) mono: 0.22924812869928668


## Optimization

### Setting up the optimization problem

bounds of the design space and starting point: derived from maximal load power

In [23]:
Pload_max = maximum(Pload)

xmin = [0., 0., 1e-3, 0.] # 1e-3 instead of 0.0, because LCOE is NaN if ther is exactly zero generation
xmax = [1.2, 10.0, 10.0, 5.0] * (Pload_max/1000)

4-element Vector{Float64}:
  2.0484
 17.07
 17.07
  8.535

Optionally ban some choices:

In [24]:
# Solar power forbidden (optional)
#xmax[3] = 1e-3
# Wind power forbidden (optional)
#xmax[4] = 0.

Check cost function on `xmin` and `xmax`

In [10]:
obj_multi(xmin), obj_multi(xmax)

((0.10149685980963595, 0.9998470957371233), (0.8229416738277807, 0.0))

### Initial point generation from Random

In [40]:
"""uniformly random vector between xmin and xmax, for a given PRNG seed"""
function x0_rng(seed, xmin, xmax)
    n = length(xmin)
    rng = MersenneTwister(seed)
    xmin + rand(rng, n).*(xmax-xmin)
end
# Test with 3 different seeds:
[x0_rng(0, xmin, xmax) x0_rng(0, xmin, xmax) x0_rng(1, xmin, xmax) x0_rng(2, xmin, xmax)]

4×4 Matrix{Float64}:
  1.68716   1.68716  0.483491   0.751346
 15.5398   15.5398   5.91505    8.94261
  2.80997   2.80997  5.3386     3.58987
  1.5135    1.5135   0.0675057  6.99305

### Wrapper of the optimization process

This is an optional step, but recommended to explore easily the impact of the many parameters taken by optimization algorithms.

See [optimization termination conditions](https://nlopt.readthedocs.io/en/latest/NLopt_Introduction/#termination-conditions) in NLopt documention for the meaning of `xtol_rel`

In [11]:
"""Optimize sizing of microgrid based on the `obj` function

Parameters:
- `x0`: initial sizing (for the algorithms which need them)
- `shed_max`: load shedding penalty threshold (same as in `obj`)
- `algo` could be one of LN_SBPLX, GN_DIRECT, GN_ESCH...
- `maxeval`: maximum allowed number of calls to the objective function,
  that is to the microgrid simulation
- `xtol_rel`: termination condition based on relative change of sizing, see NLopt doc.
- `srand`: random number generation seed (for algorithms which use some stochastic search)

Problem bounds are taken as the global variables `xmin`, `xmax`,
but could be added to the parameters as well.
"""
function optim_mg(x0, shed_max, algo=:LN_SBPLX, maxeval=1000, xtol_rel=1e-4, srand=1)
    nx = length(x0) # number of optim variables
    opt = Opt(algo, nx)
    NLopt.srand(srand)
    
    opt.lower_bounds = xmin
    opt.upper_bounds = xmax
    opt.min_objective = (x, grad) -> obj(x, grad, shed_max)
    opt.xtol_rel = xtol_rel
    opt.maxeval = maxeval
    
    (fopt, xopt, ret) = optimize(opt, x0)
    return fopt, xopt, ret, opt.numevals
end

optim_mg

### Run optimization & analyze results

In [12]:
maxeval=1000
xtol_rel = 1e-4

0.0001

In [59]:
algo_list = [:GN_CRS2_LM, :GN_DIRECT, :GN_ESCH, :LN_SBPLX]
srand_list = collect(1:5)
maxeval_list = [10, 20, 30, 60] # close to 10^(1/4) ~ 1.8 increments
maxeval_list = [maxeval_list; maxeval_list*10]
shed_max_list = [0., 0.01]

2-element Vector{Float64}:
 0.0
 0.01

Short version:

In [14]:
algo_list = [:GN_CRS2_LM, :LN_SBPLX]
srand_list = collect(1:2)
maxeval_list = [100, 1000] # close to 10^(1/4) ~ 1.8 increments
shed_max_list = [0., 0.01]

2-element Vector{Float64}:
 0.0
 0.01

Initial starting points for each seed `srand`:

In [60]:
[x0_rng(srand, xmin, xmax) for srand in srand_list]

5-element Vector{Vector{Float64}}:
 [0.4834907052541369, 5.915045432256765, 5.338595242528336, 0.06750573373843591]
 [0.751345771241941, 8.942607271569909, 3.589867014821392, 6.993050986142526]
 [1.6626830127935228, 16.87254047910793, 13.78630141409919, 8.279725362491114]
 [1.393074306890772, 14.926637765423816, 15.773952313776764, 7.931884627830944]
 [1.0863989706781596, 13.263546529065476, 5.144066424643939, 7.522314745748859]

Running the experiment, [constructing a DataFrame Row by Row](https://dataframes.juliadata.org/stable/man/getting_started/#Constructing-Row-by-Row)

In [61]:
df = DataFrame(
    shed_max=Float64[], algo=String[], srand=Int[], maxeval=Int[],
    numevals=Int[], ret=String[], topt=Float64[],
    fopt=Float64[], lcoe_opt=Float64[], shed_rate_opt=Float64[],
    xopt_gen=Float64[], xopt_sto=Float64[], xopt_pv=Float64[], xopt_wind=Float64[]
)

Row,shed_max,algo,srand,maxeval,numevals,ret,topt,fopt,lcoe_opt,shed_rate_opt,xopt_gen,xopt_sto,xopt_pv,xopt_wind
,Float64,String,Int64,Int64,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [62]:
t_exp = @elapsed for shed_max in shed_max_list # problem type
    @printf("shed_max: %.3f\n", shed_max)
    for maxeval in maxeval_list, algo in algo_list, srand in srand_list # optimizer settings
        x0 = x0_rng(srand, xmin, xmax)
        @printf("maxeval=%d, algo: %s, srand=%d: ", maxeval, algo, srand)
        topt = @elapsed fopt, xopt, ret, numevals = optim_mg(x0, shed_max, algo, maxeval, xtol_rel, srand)
        @printf("%.6f in %.1f s\n", fopt, topt)
        # round to 1/100th of kW
        xopt = round.(xopt*1000; digits=2) # kW
        # compute multiobjective performance
        lcoe_opt, shed_rate_opt = obj_multi(xopt)
        push!(df, (
                shed_max, String(algo), srand, maxeval,
                numevals, String(ret), topt,
                fopt, lcoe_opt, shed_rate_opt,
                xopt[1], xopt[2], xopt[3], xopt[4]     
        ))
    end
end
@printf("Experiment run in %.1f s\n", t_exp)

shed_max: 0.000
maxeval=10, algo: GN_CRS2_LM, srand=1: 0.375415 in 0.0 s
maxeval=10, algo: GN_CRS2_LM, srand=2: 0.363167 in 0.0 s
maxeval=10, algo: GN_CRS2_LM, srand=3: 0.264891 in 0.0 s
maxeval=10, algo: GN_CRS2_LM, srand=4: 0.430831 in 0.0 s
maxeval=10, algo: GN_CRS2_LM, srand=5: 0.239529 in 0.0 s
maxeval=10, algo: GN_DIRECT, srand=1: 0.400483 in 0.0 s
maxeval=10, algo: GN_DIRECT, srand=2: 0.400483 in 0.0 s
maxeval=10, algo: GN_DIRECT, srand=3: 0.400483 in 0.0 s
maxeval=10, algo: GN_DIRECT, srand=4: 0.400483 in 0.0 s
maxeval=10, algo: GN_DIRECT, srand=5: 0.400483 in 0.0 s
maxeval=10, algo: GN_ESCH, srand=1: 0.338473 in 0.0 s
maxeval=10, algo: GN_ESCH, srand=2: 211.388451 in 0.0 s
maxeval=10, algo: GN_ESCH, srand=3: 0.253418 in 0.0 s
maxeval=10, algo: GN_ESCH, srand=4: 0.829003 in 0.0 s
maxeval=10, algo: GN_ESCH, srand=5: 10.051071 in 0.0 s
maxeval=10, algo: LN_SBPLX, srand=1: 0.331165 in 0.0 s
maxeval=10, algo: LN_SBPLX, srand=2: 0.456404 in 0.0 s
maxeval=10, algo: LN_SBPLX, srand=3:

In [63]:
df

Row,shed_max,algo,srand,maxeval,numevals,ret,topt,fopt,lcoe_opt,shed_rate_opt,xopt_gen,xopt_sto,xopt_pv,xopt_wind
,Float64,String,Int64,Int64,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,GN_CRS2_LM,1,10,10,MAXEVAL_REACHED,0.0018255,0.375415,144.911,0.0,1795.2,15270.9,1452.62,333.33
2,0.0,GN_CRS2_LM,2,10,10,MAXEVAL_REACHED,0.00191205,0.363167,319.591,0.0,1749.28,8436.62,14451.0,679.77
3,0.0,GN_CRS2_LM,3,10,10,MAXEVAL_REACHED,0.0019251,0.264891,229.744,0.0,1829.11,15299.7,2144.62,1768.82
4,0.0,GN_CRS2_LM,4,10,10,MAXEVAL_REACHED,0.00218563,0.430831,420.609,0.0,1767.76,16786.7,2797.62,5098.25
5,0.0,GN_CRS2_LM,5,10,10,MAXEVAL_REACHED,0.00372152,0.239529,123.071,0.0,1966.93,3216.24,415.89,1745.88
6,0.0,GN_DIRECT,1,10,10,MAXEVAL_REACHED,0.00855499,0.400483,375.534,0.0,1707.0,2845.0,8535.5,4267.5
7,0.0,GN_DIRECT,2,10,10,MAXEVAL_REACHED,0.00173695,0.400483,375.534,0.0,1707.0,2845.0,8535.5,4267.5
8,0.0,GN_DIRECT,3,10,10,MAXEVAL_REACHED,0.00176776,0.400483,375.534,0.0,1707.0,2845.0,8535.5,4267.5
9,0.0,GN_DIRECT,4,10,10,MAXEVAL_REACHED,0.0019761,0.400483,375.534,0.0,1707.0,2845.0,8535.5,4267.5


**TODO**: post process fopt to have it log of relative difference to best

## Export

In [64]:
using CSV

In [65]:
CSV.write("df_320.csv", df)

"df_320.csv"